# OpenPoseでS3の画像を処理する

S3に置かれている一連の画像ファイルを [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) で処理します。処理した画像ファイルは S3 に保存します。

このNotebookではVCノードとして起動しているAWSのGPUインスタンスを使用してS3の画像ファイルをOpenPoseで処理します。S3の画像ファイルの取得、保存には[goofys](https://github.com/kahing/goofys) を利用してファイルシステムとしてアクセスします。

# 事前チェック

このNotebookの処理は「05-001-GPUインスタンスを起動する.ipynb」で起動したGPUインスタンスを利用することを想定しています。

それ以外のノードに対して環境構築を行う場合は、以下の条件を満たすように準備を行ってください。

1. 対象となるノードを Ansible で操作できる
1. GPUが利用可能であること
1. [nvidia-docker](https://github.com/NVIDIA/nvidia-docker)のセットアップが済んでいること
1. 構築に利用するツールがインストールされていること
  - rsync
  - docker-compose
1. goofs がセットアップされていること

GPUインスタンスのVCノードを Ansible で操作できることを確認します。

In [ ]:
target = 'openpose-05'

In [ ]:
!ansible {target} -m ping

GPUが利用できることを確認します。`/dev/nvidia0` があることを確認します。

In [ ]:
!ansible {target} -a 'ls -la /dev/nvidia0'

Docker のランタイムとして nvidia が指定できることを確認します。

nvidia のランタイムが登録されている場合は、以下のような出力となります。
```
{nvidia-container-runtime []}
```

In [ ]:
!ansible {target} -a 'docker info -f "{{%raw%}}{{{{.Runtimes.nvidia}}}}{{%endraw%}}"'

必要なコマンドがインストールされていることを確認します。

In [ ]:
!ansible {target} -a 'which rsync'

In [ ]:
!ansible {target} -a 'docker-compose version'

[FUSE](https://github.com/libfuse/libfuse) のセットアップが済んでいて `/dev/fuse`が存在していることを確認します。

In [ ]:
!ansible {target} -a 'ls -la /dev/fuse'

# パラメータの設定

## S3 に関するパラメータの指定

処理対象となるS3のバケット名を指定してください。

In [ ]:
s3_bucket = 'nii-dp-test-20190220'

処理対象となるバケットにおけるパスを指定してください。

In [ ]:
s3_src_path = 'topics/distributed-video1/year=2019/month=03/day=29'

処理結果の画像を格納するバケットにおけるパスを指定してください。

In [ ]:
s3_dest_path = 'openpose/distributed-video1/year=2019/month=03/day=29'

AWS S3にアクセスするためのアクセスキーとシークレットキーを入力してください。

In [ ]:
from getpass import getpass

In [ ]:
aws_access_key = getpass()

In [ ]:
aws_secret_key = getpass()

## 処理対象の絞り込みに関するパラメータ

処理スクリプトの引数を格納するリストを初期化します。

In [ ]:
openpose_opts = []

処理対象となる画像数の上限を設定する場合は次のセルを実行してください。

> 700枚の画像を処理するのに１分半～２分程度かかります。

In [ ]:
openpose_opts.extend([
    '-M',
    '700',        # OpenPoseで処理する画像数の上限枚数
])

処理対象とする画像ファイルを、ある時刻以降のものだけに限定する場合は次のセルで値の指定を行い実行してください。

> 時刻の比較は画像ファイルのタイムスタンプとの比較になります。Kafkaのトピックに送った時刻とは異なりますので気を付けてください。また、指定した時刻は `JST` とみなします。

In [ ]:
openpose_opts.extend([
    '-T',
    '2019/03/29 0:00',
])

OpenPoseの対象となる画像ファイルを、数枚に１枚の割合のみにする場合は、次のセルのコメントを外してを実行してください。

In [ ]:
# openpose_opts.extend([
#    '-D',
#    '3',        # この場合、画像3枚あたり1枚だけ OpenPoseで処理して、あとは捨てる
# ])

# OpenPoseの実行

## 設定ファイルの配置

In [ ]:
params = {
    'registry': '192.168.2.1:5001',
    's3_bucket': s3_bucket,
    'script_opts': [
        '-s', '/mnt/s3/' + s3_src_path,
        '-d', '/mnt/s3/' + s3_dest_path,
    ],
    's3_access_key': aws_access_key,
    's3_secret_key': aws_secret_key,
}

if 'openpose_opts' in locals():
    params['script_opts'].extend(openpose_opts)

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import json

storage = 's3'
op_dir = storage + '/openpose-offline'
!ansible {target} -m synchronize -a 'src=openpose/openpose-offline dest=./{storage}/'
with TemporaryDirectory() as work_dir:
    vars_file = Path(work_dir) / 'vars.json'
    with vars_file.open(mode='w') as f:
        json.dump(params, f)
    !ansible {target} -m template -e @{vars_file} \
        -a 'src=openpose/openpose-offline/docker-compose.yml dest={op_dir}/'
    !ansible {target} -m template -e storage={storage} -e @{vars_file} \
        -a 'src=openpose/openpose-offline/.env dest={op_dir}/'

## OpenPoseの実行

S3 の画像ファイルをOpenPoseで処理します。

まず、OpenPoseのコンテナイメージを取得します。

In [ ]:
!ansible {target} -a 'chdir={op_dir} docker-compose pull'

OpenPoseの処理を行うコンテナを起動して、処理を行います。

In [ ]:
!ansible {target} -a 'chdir={op_dir} docker-compose up'

# 処理結果の確認

OpenPose で処理した一連の画像を動画に変換して表示します。

## パラメータの指定

画像ファイルのあるディレクトリを指定してください。

In [ ]:
image_dir = s3_dest_path
print(image_dir)

動画のサイズを指定してください。

In [ ]:
#width = 320
#height = 200
width = 640
height = 400

動画のフレームレートを指定してください。

In [ ]:
rate = 10.0

処理対象とする画像ファイル数の上限を指定してください。

In [ ]:
max_images = 1000

処理対象にある画像ファイルのうち、スキップする画像数を指定してください。

In [ ]:
skip_images = 200

## 動画の作成

設定ファイルをGPUインスタンスに配置します。

In [ ]:
params = {
    'registry': '192.168.2.1:5001',
    's3_bucket': s3_bucket,
    'script_opts': [
        '-s', '/mnt/s3/' + image_dir,
        '-W', width,
        '-H', height,
        '-R', rate,
        '-M', max_images,
        '-S', skip_images,
    ],
    's3_access_key': aws_access_key,
    's3_secret_key': aws_secret_key,
}

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path
import json

ffmpeg_dir = storage + '/ffmpeg'
!ansible {target} -m synchronize -a 'src=ffmpeg dest=./{storage}'
with TemporaryDirectory() as work_dir:
    vars_file = Path(work_dir) / 'vars.json'
    with vars_file.open(mode='w') as f:
        json.dump(params, f)
    !ansible {target} -m template -e @{vars_file} \
        -a 'src=ffmpeg/docker-compose.yml dest={ffmpeg_dir}/'
    !ansible {target} -m template  -e storage={storage} -e @{vars_file} \
        -a 'src=ffmpeg/.env dest={ffmpeg_dir}/'

動画作成を行うコンテナを起動します。

> 1500枚の画像から動画を作成するのに１分弱かかります。

In [ ]:
!ansible {target} -a 'chdir={ffmpeg_dir} docker-compose pull'

In [ ]:
!ansible {target} -a 'chdir={ffmpeg_dir} docker-compose up'

動画ファイルをNotebook環境に取得します。

In [ ]:
from datetime import datetime

mov_file = f'mp4/openpose-{datetime.now().strftime("%Y%m%d%H%M%S")}.mp4'
!mkdir -p mp4
!ansible {target} -m fetch -a 'src={ffmpeg_dir}/output/openpose.mp4 dest={mov_file} flat=yes'

## 動画の表示

OpenPoseで処理した画像から作成した動画を表示します。

> 処理対象とする画像のスキップ数を大きくしすぎると、処理対象となる画像が無くなり動画が表示されません。

In [ ]:
from IPython.display import Video
Video(mov_file)